In [2]:
import cv2

In [7]:
import pytesseract

In [8]:
pytesseract.pytesseract.tesseract_cmd = r'/usr/bin/tesseract'



In [118]:
from PIL import Image 
import pytesseract 
import sys 
from pdf2image import convert_from_path 
import os 
from langdetect import detect

# Path of the pdf 
PDF_file = "100.pdf"
  
''' 
Part #1 : Converting PDF to images 
'''
  
# Store all the pages of the PDF in a variable 
pages = convert_from_path(PDF_file, 500) 
  
# Counter to store images of each page of PDF to image 
image_counter = 1
  
# Iterate through all the pages stored above 
for page in pages: 
  
    # Declaring filename for each page of PDF as JPG 
    #filename = "page_"+str(image_counter)+".jpg"
      
    # Save the image of the page in system 
    #page.save(filename, 'JPEG') 
  
    # Increment the counter to update filename 
    #image_counter = image_counter + 1
  
    ''' 
    Part #2 - Recognizing text from the images using OCR 
    '''
    
    # Variable to get count of total number of pages 
    #filelimit = image_counter-1
    
    # Creating a text file to write the output 
    #outfile = "out_text.txt"
    
    # Open the file in append mode so that  
    # All contents of all images are added to the same file 
    #f = open(outfile, "a") 

    fr_list = []
    
    # Iterate from 1 to total number of pages 
    #for i in range(1, filelimit + 1): 
  
    # Set filename to recognize text from 
    
    #filename = "page_"+str(i)+".jpg"
          
    # Recognize the text as string in image using pytesserct 
    text = str(pytesseract.image_to_string(page)) 
  
    text = text.replace('-\n', '')

    def text_to_list(text):
        to_list = []
        to_list = text.split('\n\n\n')
        to_list = list(filter(lambda x: x != " ", to_list))
    
        return to_list


    def is_french(stringx) :
        try :
            if detect(stringx.lower()) == 'fr' :
                return True
        except :
                return False

    text_list = text_to_list(text)
    for line in text_list :
        if is_french(line) :
            fr_list.append(line.replace('\n',' '))
    

  
    # Finally, write the processed text to the file. 
    #f.write(text) 
  
# Close the file after writing all the text. 
#f.close() 

In [119]:
fr_list

['Chapitre V — Paix sociale',
 'Les parties s’engagent a préserver la paix sociale en 2019-2020 et a ne formuler pendant cette période aucune exigence complémentaire au niveau sectoriel ou au niveau de |’entreprise. Chapitre VI — Durée',
 'La présente convention collective de travail est conclue pour une durée indéterminée et entre en vigueur le 1% novembre 2019.',
 'Elle peut étre résiliée par une des parties moyennant un préavis de 3 mois adressé par lettre recommandée au président de la Commission Paritaire Auxiliaire pour Ouvriers.']

In [13]:
img = cv2.imread("page_1.jpg")
text = pytesseract.image_to_string(img)
print(text)

 

COLLECTIEVE ARBEIDSOVEREENKOMST VAN 13
NOVEMBER 2019 GESLOTEN IN HET
AANVULLEND PARITAIR COMITE VOOR DE
WERKLIEDEN BETREFFENDE HET SECTORAAL
AKKOORD 2019-2020 — KOOPKRACHT

 

Hoofdstuk | Toepassingsgebied

Artikel 1

Deze collectieve arbeidsovereenkomst is van
toepassing op de werkgevers en de arbeiders van
de ondernemingen die onder de bevoegcheid
vallen van het Aanvullend Paritair Comité voor de
Werklieden. Onder “arbeiders” wordt verstaan
de mannelijke en vrouwelijke arbeiders.
Hoofdstuk II — Algemene bepaling

Artikel 2

Deze collectieve arbeidsovereenkomst is gesloten
in het kader van het Koninklijk besluit van 19 april
2019 tot uitvoering van artikel 7§1 van de Wet
van 26 juli 1996 ter bevordering van de
werkgelegenheid en en tot preventieve vrijwaring
van het concurrentievermogen.

Hoofdstuk II] — verhoging van minimumuurloon
en van effectief uurloon

Artikel 3

Het minimumuurloon en het effectieve uurloon
worden met 1,1% verhoogd vanaf 1/11/2019.

De verhoging van het effec

In [19]:
image = cv2.imread("page_1.jpg")
rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
# OCR the image, supplying the country code as the language parameter
config = "--psm 3"
text = pytesseract.image_to_string(image,config=config, lang="fra")

print(text)

Neerlegging-Dépôt: 27/11/2019 …
Regist.-Enregistr.: 16/12/2019
N°: 155913/CO/100

 

COLLECTIEVE ARBEIDSOVEREENKOMST VAN 13 | CONVENTION COLLECTIVE DE TRAVAILDU13

 

NOVEMBER 2019 GESLOTEN IN HET NOVEMBRE 2019 CONCLUE AU SEIN DE LA
AANVULLEND PARITAIR COMITÉ VOOR DE COMMISSION PARITAIRE AUXILIAIRE POUR
WERKLIEDEN BETREFFENDE HET SECTORAAL OUVRIERS RELATIVE À L'ACCORD SECTORIEL
AKKOORD 2019-2020 — KOOPKRACHT 2019-2020 — POUVOIR D'ACHAT

Hoofdstuk I Toepassingsgebied Chapitre | Champ d’application

 

Artikel Article 1

Deze collectieve arbeidsovereenkomst is van La présente convention collective de travail
toepassing op de werkgevers en de arbeiders van | s'applique aux employeurs et aux ouvriers des

de ondernemingen die onder de bevoegdheid entreprises relevant de la compétence de la
vallen van het Aanvullend Paritair Comité voor de | Commission Paritaire Auxiliaire pour Ouvriers.
Werklieden. Onder “arbeiders” wordt verstaan Par « ouvriers », on entend les ouvriers masculins
de manne

In [108]:
from langdetect import detect


def is_french(stringx) :
    try :
        if detect(stringx.lower()) == 'fr' :
            return True
    except :
            return False

fr_list = []
for line in text_list :
    if is_french(line) :
        fr_list.append(line.replace('\n',' '))
        
    

 

#for line in mixed_text.strip().splitlines():
 # print(line + u"\n")
 # for language in Detector(line).languages:
  #  print(language)
 # print("\n")
 

In [56]:
is_french(text_list[12].lower())

True

In [113]:
fr_list[9]

'Le montant des autres avantages doit étre vérifiable.'

In [93]:
print(text_list[12])

CONVENTION COLLECTIVE DE TRAVAILDU13
NOVEMBRE 2019 CONCLUE AU SEIN DELA
COMMISSION PARITAIRE AUXILIAIRE POUR
OUVRIERS RELATIVE A L’ACCORD SECTORIEL
2019-2020 — POUVOIR D’ACHAT
